In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import cv2
import re
from sklearn.model_selection import train_test_split

In [ ]:
index_file = pd.read_csv('../input/chinese-mnist/chinese_mnist.csv')
index_file

In [ ]:
labels = sorted(index_file.value.unique())
labels

In [ ]:
data = []
width = 60
height = 60

for dirname, _, filenames in os.walk('/kaggle/input/chinese-mnist/data/data/'):
    for filename in filenames:
        comb = re.findall('[0-9]+', filename)
        comb = [int(i) for i in comb]
        if(len(comb)==3):
            label = labels.index(index_file[index_file.suite_id==comb[0]][index_file.sample_id==comb[1]][index_file.code==comb[2]].value.values[0])
            image_data = cv2.imread(os.path.join(dirname, filename), cv2.IMREAD_GRAYSCALE)
            resized_data = cv2.resize(image_data, (width, height))
            data.append([resized_data, label])
        else:
            print('Incompatible file format')
            break; 

In [ ]:
plt.figure(figsize=(19, 16))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cv2.cvtColor(data[i][0], cv2.COLOR_BGR2RGB))
    plt.xlabel(labels[data[i][1]])

In [ ]:
X = [] 
y = [] 

for feature, label in data:
    X.append(feature)
    y.append(label)

X = np.array(X).reshape(-1, width, height, 1)
y = np.array(y)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=69)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
model = Sequential()

model.add(Conv2D(256, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=1))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=1))

model.add(Flatten())

model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization(axis=1))

model.add(Dense(len(labels)))
model.add(Activation('softmax'))

early_stop = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=70, epochs=5, validation_split=0.15, callbacks=[early_stop])

In [ ]:
model.evaluate(X_test, y_test)